In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt

import wandb
wandb.login()

# # Cargar los datos desde el CSV, ignorando la columna de índice
file_path = 'C:\\Users\\48242450\\Documents\\Pruebas-Proyecto\\conversion_predictors_of_clinically_isolated_syndrome_to_multiple_sclerosis (2).csv'
df = pd.read_csv(file_path, index_col=0)
# df = pd.read_csv('./data.csv')

# Verificar los nombres de las columnas
print(df.columns)

# Revisar si hay valores faltantes en el dataframe
print(df.isna().sum())

# Imputar valores faltantes
imputer = SimpleImputer(strategy='mean')
X = df.drop(['group', 'Initial_EDSS', 'Final_EDSS'], axis=1)
y = df['group']

# Imputar valores faltantes en X
X_imputed = imputer.fit_transform(X)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, random_state=42)

# Definir el modelo SVM y los parámetros para la búsqueda en GridSearchCV
svm_model = SVC(probability=True)

# Parámetros para explorar
param_grid = {
    'kernel': ['linear', 'rbf', 'poly'],
    'C': [0.1, 1, 10, 100],  # Regularización
    'gamma': ['scale', 'auto'],  # Parámetro del kernel RBF y polinómico
    'degree': [3, 4, 5]  # Solo se aplica al kernel polinomial
}

# Implementar GridSearchCV
grid_search = GridSearchCV(svm_model, param_grid, cv=5, scoring='roc_auc', verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Mejor modelo encontrado
best_model = grid_search.best_estimator_
print(f"Mejores parámetros: {grid_search.best_params_}")

# Hacer predicciones con el mejor modelo
y_pred = best_model.predict(X_test)
y_prob = best_model.predict_proba(X_test)[:, 1]

# Evaluar el modelo
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Calcular ROC AUC
fpr, tpr, _ = roc_curve(y_test, y_prob, pos_label=best_model.classes_[1])
roc_auc = roc_auc_score(y_test, y_prob)

print(f'ROC AUC: {roc_auc:.2f}')

# Graficar la curva ROC
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc='lower right')
plt.show()


In [ ]:
from sklearn.metrics import accuracy_score

# Calcular la precisión en el conjunto de entrenamiento y prueba
train_accuracy = accuracy_score(y_train, best_model.predict(X_train))
test_accuracy = accuracy_score(y_test, y_pred)

print(f"Precisión en el conjunto de entrenamiento: {train_accuracy:.2f}")
print(f"Precisión en el conjunto de prueba: {test_accuracy:.2f}")

### RandomizeSearchCV

In [ ]:
from sklearn.model_selection import  RandomizedSearchCV
from scipy.stats import uniform, randint




# # Cargar los datos desde el CSV, ignorando la columna de índice
file_path = 'C:\\Users\\48242450\\Documents\\Pruebas-Proyecto\\conversion_predictors_of_clinically_isolated_syndrome_to_multiple_sclerosis (2).csv'
df = pd.read_csv(file_path, index_col=0)

# Verificar los nombres de las columnas
print(df.columns)

# Revisar si hay valores faltantes en el dataframe
print(df.isna().sum())

# Imputar valores faltantes
imputer = SimpleImputer(strategy='mean')
X = df.drop(['group', 'Initial_EDSS', 'Final_EDSS'], axis=1)
y = df['group']

# Imputar valores faltantes en X
X_imputed = imputer.fit_transform(X)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, random_state=42)

# Definir el modelo SVM y los parámetros para la búsqueda en RandomizedSearchCV
svm_model = SVC(probability=True)

# Parámetros para explorar con RandomizedSearchCV
param_dist = {
    'kernel': ['linear', 'rbf', 'poly'],
    'C': uniform(0.1, 100),  # Distribución uniforme para la regularización
    'gamma': ['scale', 'auto'],  # Parámetro del kernel RBF y polinómico
    'degree': randint(3, 6)  # Solo se aplica al kernel polinomial
}

# Implementar RandomizedSearchCV con mayor verbosidad
random_search = RandomizedSearchCV(svm_model, param_distributions=param_dist, n_iter=100, cv=5, scoring='roc_auc', verbose=3, random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)

# Mejor modelo encontrado
best_model = random_search.best_estimator_
print(f"Mejores parámetros: {random_search.best_params_}")

# Hacer predicciones con el mejor modelo
y_pred = best_model.predict(X_test)
y_prob = best_model.predict_proba(X_test)[:, 1]

# Evaluar el modelo
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Calcular ROC AUC
fpr, tpr, _ = roc_curve(y_test, y_prob, pos_label=best_model.classes_[1])
roc_auc = roc_auc_score(y_test, y_prob)

print(f'ROC AUC: {roc_auc:.2f}')

# Graficar la curva ROC
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc='lower right')
plt.show()


20 casos verdaderamente positivos.
6 falsos negativos.
8 falsos positivos.
21 verdaderamente negativos.


### HyperOpt

In [ ]:
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from hyperopt.pyll.base import scope


# # Cargar los datos desde el CSV, ignorando la columna de índice
# file_path = 'C:\\Users\\48242450\\Documents\\Pruebas-Proyecto\\conversion_predictors_of_clinically_isolated_syndrome_to_multiple_sclerosis (2).csv'
# df = pd.read_csv(file_path, index_col=0)

# Imputar valores faltantes
imputer = SimpleImputer(strategy='mean')
X = df.drop(['group', 'Initial_EDSS', 'Final_EDSS'], axis=1)
y = df['group']
X_imputed = imputer.fit_transform(X)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, random_state=42)

# Definir el espacio de búsqueda para HyperOpt
space = {
    'kernel': hp.choice('kernel', ['linear', 'rbf', 'poly']),
    'C': hp.uniform('C', 0.1, 100),
    'gamma': hp.choice('gamma', ['scale', 'auto']),
    'degree': scope.int(hp.quniform('degree', 3, 5, 1))  # Asegurarse de que 'degree' sea un entero
}

# Función objetivo para minimizar
def objective(params):
    # Si 'kernel' es 'poly', asegúrate de que 'degree' sea relevante
    if params['kernel'] == 'poly' and 'degree' not in params:
        params['degree'] = 3  # O asigna un valor predeterminado si 'degree' no está en los parámetros

    svm_model = SVC(probability=True, **params)
    svm_model.fit(X_train, y_train)
    y_prob = svm_model.predict_proba(X_test)[:, 1]
    roc_auc = roc_auc_score(y_test, y_prob)
    return {'loss': -roc_auc, 'status': STATUS_OK}

# Ejecutar HyperOpt
trials = Trials()
best_params = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=100, trials=trials, rstate=None)

# Convertir 'degree' a entero si es necesario
best_params['degree'] = int(best_params['degree'])

# Si 'kernel' es un índice en lugar del valor, asigna el valor correcto
kernel_options = ['linear', 'rbf', 'poly']
best_params['kernel'] = kernel_options[best_params['kernel']]

# Ajustar el modelo con los mejores parámetros
svm_model = SVC(probability=True, **best_params)
svm_model.fit(X_train, y_train)

# Hacer predicciones con el mejor modelo
y_pred = svm_model.predict(X_test)
y_prob = svm_model.predict_proba(X_test)[:, 1]

# Evaluar el modelo
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Calcular ROC AUC
fpr, tpr, _ = roc_curve(y_test, y_prob, pos_label=svm_model.classes_[1])
roc_auc = roc_auc_score(y_test, y_prob)
print(f'ROC AUC: {roc_auc:.2f}')

# Graficar la curva ROC
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc='lower right')
plt.show()


In [ ]:
def train_and_evaluate_svm(params):

    # Crear y ajustar el modelo SVM
    svm_model = SVC(probability=True, **params)
    svm_model.fit(X_train, y_train)

    # Hacer predicciones
    y_pred = svm_model.predict(X_test)
    y_prob = svm_model.predict_proba(X_test)[:, 1]

    # Evaluar el modelo
    confusion_matrix_df = pd.DataFrame(confusion_matrix(y_test, y_pred))
    classification_report_df = pd.DataFrame(
        classification_report(y_test, y_pred, output_dict=True)
    )

    # Calcular métricas
    roc_auc = roc_auc_score(y_test, y_prob)
    accuracy = accuracy_score(y_test, y_pred)

    # Registrar métricas en Wandb
    wandb.log(
        {
            "confusion_matrix": wandb.Table(dataframe=confusion_matrix_df),
            "classification_report": wandb.Table(dataframe=classification_report_df),
            "roc_auc": roc_auc,
            "accuracy": accuracy,
        }
    )

    # Calcular la curva ROC
    fpr, tpr, _ = roc_curve(y_test, y_prob, pos_label=svm_model.classes_[1])

    # Registrar la curva ROC como imagen
    plt.figure()
    plt.plot(fpr, tpr, color="darkorange", lw=2, label="ROC curve (area = %0.2f)" % roc_auc)
    plt.plot([0, 1], [0, 1], color="navy", lw=2, linestyle="--")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("Receiver Operating Characteristic (ROC)")
    plt.legend(loc="lower right")
    plt.tight_layout() 

    wandb.log({"roc_curve": plt})